In [1]:
from create_fashion_items_image import create_cw_image, create_fashion_items_image
from embedding_word import get_embedding_word
from fashion_class.FashionItem import FashionItem
from fashion_class.ImageStruct import ImageStruct
from get_caption_model import get_caption_model
from search_items import search_items_from_caption_embedding

In [2]:
# modelを定義
print('model初期化')
caption_model, tokenizer = get_caption_model()
print('model初期化完了')

print('データセット初期化中')
# データセットを初期化
annotation_file, tensor_file = \
      './data/anotation_new.csv', './data/resnet50_tensor.pt'
dataset = ImageStruct(annotation_file, tensor_file, init_item_length=300)
print('データセット初期化終了')


model初期化


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model初期化完了
データセット初期化中
【================================================================================================    】データセット初期化終了


In [3]:
# ユーザに表示して、気に入ったアイテムをいくつか選んでもらう
category_list = ["tops", "bottoms", "shoes"]
category_dataset = {
    'tops': dataset.tops,
    'bottoms': dataset.bottoms,
    'shoes': dataset.shoes
}
required_items = {}
initial_items = {}
for c in category_list:
    required_items[c] = []
    # 好きな単語をいくつか入力してもらう
    favorite_word = input(f"{c} を選びます。単語をスペース区切りで入力してください: ")
    print(f'{favorite_word} に基づき、画像検索を行います。')
    # captionのベクトル化を行い、それに合う画像を検索
    caption_embedding = get_embedding_word(favorite_word, caption_model, tokenizer)

    # imagesは、このような形式
    # (-score, t_img_path, t_id)
    fashion_items = search_items_from_caption_embedding(caption_embedding, category_dataset[c], limit=100)

    image = create_fashion_items_image(fashion_items)
    image.show()
    favorite_item_set = list(map(int, input(f'気に入った {c} アイテムのidをスペース区切りで入力してください').split()))
    for i in favorite_item_set:
        # TODO: エラーハンドリングしないと落ちる
        required_items[c].append(fashion_items[i-1])
    if len(required_items[c]) == 0:
        # マジックナンバー
        initial_items[c] = fashion_items[:4]
    else:
        initial_items[c] = []



 に基づき、画像検索を行います。
[          ] 0.0% に基づき、画像検索を行います。
[          ] 0.0% に基づき、画像検索を行います。
[          ] 0.0%

In [40]:
import importlib

import fashion_class
from fashion_class import CapsuleWardrobeClass
from fashion_class.CapsuleWardrobeClass import CapsuleWardrobe
import CreateCW
importlib.reload(CreateCW)
importlib.reload(fashion_class)
from fashion_class.FashionItem import FashionItem
importlib.reload(CapsuleWardrobeClass)
from CreateCW import change_item_recommandation, create_cw, search_alternate_item

In [41]:

# CWを構築
print("与えられた情報をもとにCWを構築しています。")

cw = create_cw(required_items, dataset, initial_items)
print("生成されたcwがこちらです")

# 画像を表示
while True:
    image = create_cw_image(cw)
    image.show()
    item = input("出来上がったcwに対して、気に入らないitemを1つ tops:2 bottoms:3 のように指定してください。CWが完成した場合はqと入力してください > ")
    if item == 'q':
        break
    kind, index = item.split(':')
    new_item = search_alternate_item(cw, kind, index, dataset)
    # 1枚多く計算する
    # 既存のアイテムは外さない
    required_items = {
        'tops': cw.tops,
        'bottoms':cw.bottoms,
        'shoes':cw.shoes
    }
    # 既存のアイテムのお気に入りの入れ替え
    required_items[kind][index-1] = new_item

    cw = create_cw(required_items, dataset, max_length=cw.max_length + 1)
    recommended_change = change_item_recommandation(cw)
    for (r, i) in recommended_change.items():
        if r == kind:
            continue
        can_change = input(f'{r}の{i+1}番目を、新しい推薦アイテムと変えるとより良いCWになりますが、変えますか？')
        # if can_change == 'yes':
        

    

与えられた情報をもとにCWを構築しています。
0回目 増加分: 1.01
0回目 増加分: 454280
生成されたcwがこちらです


In [31]:
image = create_cw_image(cw)
image.show()

In [42]:
[i.item_id for i in cw.get_bottoms()]

[7686717, 40450511, 40450511, 40450511]

In [43]:
[i.item_id for i in cw.get_shoes()]

[34135184, 2619111, 2619111, 3005645]

In [44]:
[i.item_id for i in cw.get_tops()]

[9446406, 4108331, 36850737, 36850737]